# Module: Metaclasses

1. Dynamically creating classes with `type`
2. What is `__new__`?
3. A basic metaclass example
4. Use Case 1: Validation
5. `__call__` and `__new__`
6. Use Case 2: Singleton
7. Lab: Registry

## Dynamically creating classes with `type`

### Let's investigate `type`

In [1]:
type(3)

int

Check `type` of string, dictionary, and even `type` itself.

In [2]:
type("Hello")

str

In [3]:
type(type)

type

- `type` is a metaclass that creates classes.
- can be used to dynamically create classes

In [5]:
Car = type('Car', (), {})

### What are the arguments?
`type(<name>, <bases>, <dct>`

- <name> specifies the class name. This becomes the __name__ attribute of the class.
- <bases> specifies a tuple of the base classes from which the class inherits. This becomes the __bases__ attribute of the class.
- <dct> specifies a namespace dictionary containing definitions for the class body. This becomes the __dict__ attribute of the class.
    
(realpython.com)

In [6]:
SUV = type('SUV', (Car,), dict(doors=4))

4

```python
class Car():
    blah = True

class SUV(Car):
    doors = 4
```

Let's instantiate an `SUV` and introspect `__class__` and `__class__.__bases__` and check the class attribute.

### Let's add more to the class namespace.

In [1]:
EBayScraper = type('EBayScraper', (), {'domain': 'ebay.com', 'get_full_url': lambda x : 'https://{}'.format(x.domain)})

### Exercise
- Instantiate an `EBayScraper` and call `get_full_url`.
- Check the bases and namespace.

## What is `__new__`?

In [2]:
class FooMeta(type):
    def __new__(cls, name, bases, dct):
        print("In __new__")
        return super().__new__(cls, name, bases, dct)


**Discuss**: Why `cls` and not `self`? Does it matter?

Execute the following class.

In [5]:
class Foo(metaclass=FooMeta):
    def __init__(self):
        print("In __init__")

Now instantiate.

In [3]:
Foo()

What is a metaclass?

## A basic metaclass example

In [33]:
class CoolMeta(type):
    def __new__(cls, name, bases, dct):
        x = super().__new__(cls, name, bases, dct)
        x.some_val = 225
        return x

In [35]:
class OtherClass(metaclass=CoolMeta):
    pass

In [6]:
OtherClass.some_val

What does this do?

## Use Case 1: Validation
Let's also take this opportunity to get famliar with the [`requests`](https://requests.readthedocs.io/en/master/) library.

In [38]:
import requests
res = requests.get('https://news.ycombinator.com/')

In [40]:
res.content

b'<html lang="en" op="news"><head><meta name="referrer" content="origin"><meta name="viewport" content="width=device-width, initial-scale=1.0"><link rel="stylesheet" type="text/css" href="news.css?OnNWsJiz21LfZewvRRkg">\n        <link rel="shortcut icon" href="favicon.ico">\n          <link rel="alternate" type="application/rss+xml" title="RSS" href="rss">\n        <title>Hacker News</title></head><body><center><table id="hnmain" border="0" cellpadding="0" cellspacing="0" width="85%" bgcolor="#f6f6ef">\n        <tr><td bgcolor="#ff6600"><table border="0" cellpadding="0" cellspacing="0" width="100%" style="padding:2px"><tr><td style="width:18px;padding-right:4px"><a href="https://news.ycombinator.com"><img src="y18.gif" width="18" height="18" style="border:1px white solid;"></a></td>\n                  <td style="line-height:12pt; height:10px;"><span class="pagetop"><b class="hnname"><a href="news">Hacker News</a></b>\n              <a href="newest">new</a> | <a href="front">past</a> | 

### Exercise
Create a class called `HackerNewsScraper` that defines a class_level attribute `url` and has a `scrape` method that simply makes a request to the url and returns the content.

In [8]:
class HackerNewsScraper():
    url = '' # implement
    
    def scrape(self):
        # Implement
        pass

In [45]:
# HackerNewsScraper().scrape()

### Exercise
Create a metaclass `ScraperMeta` that checks if `url` is in the `dct` namespace and if it's not there, it should raise an `Exception`

In [9]:
class ScraperMeta(type):
    def __new__(cls, name, bases, dct):
        # Implement here
        pass

In [12]:
class HackerNewsScraper(metaclass=ScraperMeta):
    url = '' # Implement

    def scrape(self):
        # Implement
        pass        

### Exercise.
Define `HackerNewsScraper` **without** the `url` and see what happens, using the meta class

## `__call__` and `__new__`

In [31]:
class Bar():
    def __init__(self):
        print("Instance created!")

    def __call__(self):
        print("Instance called via special method")

In [34]:
bar = Bar()

Instance created!


In [36]:
bar()

Instance called via special method


### What is going on here?
Let's generalize the above.

1. On the metaclass, `__new__` results in the creation of the class. 
2. The `__call__` on the metaclass is called whenever the class is called for instantiation.

In [58]:
class BarMeta(type):
    def __call__(cls):
        print("Whoa! We are in __call__ of BarMeta")
        super().__call__()

    def __new__(cls, *args):
        print("Calling __new__ of BarMeta!")
        return super().__new__(cls, *args)

In [66]:
class Bar(metaclass=BarMeta):
    def __init__(self):
        print("In __init__ of Bar!")

In [69]:
Bar()

## Use Case 2: Singleton

```python
class SingletonA(metaclass=SingletonMeta):
    pass

>>> A = SingletonA()
>>> B = SingletonA()
>>> # A is B. 
```

Fill in the logic below

In [75]:
class SingletonMeta(type):
    _instance_map = {}

    def __call__(cls, name, bases, dct):
        # Implement logic here
        # If we don't have an instance already, for class `cls`
        # then call the `__call__` function to instantiate 
        # and save and return the instance
        # Otherwise, return the instance we have already created
        pass

### Exercise
- Create a class that uses `SingletonMeta` as its metaclass. 
- What happens when you try to instantiate multiple instances?
- Now create another class that uses `SingletonMeta`. Does it interfere with the previous class?

## (Optional) Lab: Registry
Use metaclasses so that any class that uses `AnimalMeta` tracks all the classes in a global `class_registry`.